## Start compute simliarity between two strings

In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
sys.path.append(project_root)
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [2]:
from src.dataset.coupert import CoupertDataset
from src.arguments import DataArguments
from utils.retrieval import retrieval
from utils.metrics import compute_recall
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from safetensors.torch import save_file, load_file

model_dir = "../model/BAAI/bge-large-en-v1.5"
embedding_path = os.path.join(
    "../embeddings/", model_dir.split("/")[-1] + ".safetensors"
)
data_dir = "../data/coupert"
device = "cuda:1"


/home/hutu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hutu/miniconda3/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
embs = load_file(embedding_path)
eval_embs = embs["eval_embs"]
gallery_embs = embs["gallery_embs"]

In [4]:
eval_df = pd.read_json("../data/coupert/coupert_eval.jsonl", lines=True)
gallery_df = pd.read_json("../data/coupert/coupert_gallery.jsonl", lines=True)


In [5]:
gallery_ids = gallery_df["dp-id"].values.tolist()
len(gallery_ids), gallery_embs.shape

(2488144, torch.Size([2488144, 1024]))

In [6]:
ret_ids = retrieval(
    eval_embs, gallery_embs, gallery_ids=gallery_ids, topK=50, device=device
)
eval_df["pred"] = ret_ids

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.49 GiB. GPU 1 has a total capacity of 10.75 GiB of which 58.50 MiB is free. Process 3132175 has 10.47 GiB memory in use. Including non-PyTorch memory, this process has 218.00 MiB memory in use. Of the allocated memory 64.00 MiB is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
eval_df["recall"] = eval_df.apply(compute_recall("pred"), axis=1)
eval_df.recall.mean()

0.8050220513713172

In [8]:
eval_df.to_json(
    "../data/coupert/coupert_eval_retrieval.jsonl", lines=True, orient="records"
)

In [ ]:
# 找出recall<0.5的, 查看label的title和预测的title
df = eval_df[eval_df.recall == 0]
for i, row in df.iterrows():
    print("=" * 80)
    print(f"id:{row['dp-id']}")
    print(row["title"])
    print(row["pred"])
    print(row["recall"])
    print(row["label"])
    label = gallery_df[gallery_df["dp-id"].isin(row["label"])]["title"].values
    pred = gallery_df[gallery_df["dp-id"].isin(row["pred"])]["title"].values
    print(label)
    print(pred)

    if i > 10:
        break

id:100880478-276
General Altimax 365AW All Weather 235/65R18 106H Passenger Tire
['63756558-0', '1944496419-0', '263856407-0', '825309229-0', '1025875951-0', '3285357502-0', '2143074354-0', '103744749-0', '1089150340-0', '2345297602-0', '1802075194-0', '1116654937-0', '859303089-0', '641605974-0', '97342521-0', '2809986687-0', '2949535540-0', '3046923580-0', '3728682553-0', '2485909690-0', '1692728746-0', '2063507715-0', '1941630913-0', '2698843653-0', '2500948563-0', '3882629832-0', '3698358263-0', '2179756829-0', '2194726494-0', '3370662454-0', '877041296-0', '3328807914-0', '2996989986-0', '1090655876-0', '3927784586-0', '914985706-0', '408978891-0', '114138937-0', '1443120372-0', '1114779548-0', '2951705214-0', '2742751943-0', '2090611530-0', '2872805129-0', '3588140544-0', '3609349826-0', '3732609234-0', '53710038-0', '988344183-0', '1869192931-0']
0.0
['1770048153-0', '1782655961-0', '4250988828-0', '592391230-0', '4111365012-0', '3437314108-0', '3807079623-0', '2044313669-0', '1